# load, process, and grid ERDDAP 

An attempt make the real-time glider data easier to work with. Once its in this format, it should be easy to grab sections using the `.sel(date = slice() )` methods of xarray. You should also have an easier time calcuating stuff, and plotting agaist time, lat or lon

This is a little fast and loose, so caveat emptor for science!

## to do:
We could(should) just convert this into a function to be run by another notebook. 
input to that function would probably be the glider dataset ID, and the variables you want

In [ ]:
import xarray as xr
import pandas as pd
from erddapy import ERDDAP
import matplotlib.pyplot as plt

import cartopy.crs as ccrs

import numpy as np

from scipy.signal import find_peaks

from scipy import stats

In [ ]:
# this creates a link to the RUCOOL server:
e = ERDDAP(
    server="http://slocum-data.marine.rutgers.edu/erddap",
    protocol="tabledap",
    response="nc",
)

# get the science data:
e.dataset_id = 'ru29-20200908T1623-profile-sci-rt'

# this connects to the data and load into an pandas dataframe
ds = e.to_pandas()
# remove the spaces from the column names
ds.columns = ds.columns.str.split(' ').str[0]

# get the time to be a datetime object
ds['time'] = pd.to_datetime(ds['time'])

# put the times in order
ds = ds.sort_values(by=['time'])

print(ds.info())


In [ ]:

# fill nans in depth for the profile breakup
interpd = ds.depth.interpolate()

plt.plot(ds.depth.values, '.')
plt.plot(interpd.values, '.')
plt.xlim([0, 1e4])



In [ ]:
# find the top and bottom of each profile
apogee, prop = find_peaks(interpd.values,  threshold=None, 
                          distance=None, prominence=50)

perogee, prop = find_peaks(-1*interpd.values,  threshold=None, 
                           distance=None, prominence=50)

# stack the index of the turning points into one vector
turns = np.sort(np.append(apogee, perogee ))
turns.shape

In [ ]:
# check your work
plt.plot(ds.time, ds.depth)
plt.plot(ds.time[apogee], ds.depth[apogee], 'o')
plt.plot(ds.time[perogee], ds.depth[perogee], 'o')


In [ ]:
# look in more detail
plt.plot( ds.depth)
plt.plot( ds.depth[apogee], 'o')
plt.plot( ds.depth[perogee], 'o')
plt.plot( ds.depth[turns])
plt.xlim([0, 1e4 ])

# plt.xlim([4.2e5, 4.5e5 ])

# GRID!

build a useful dataset for analysis. We will grid some stuff in time and depth, then grid some 1-d stuff to make coordinates and put it all together into an xarray dataset

## start with '2D' gridding

here we are going to bin (grid) stuff that has 2 dimentions: time and depth

for example: temp, salin, ...

In [ ]:
# this is your depth grid, you can set:
zgrd = np.arange(0,1000,5)

# list of variables to grid in 2d:
# you choose from the columns of the science data
# this should be the input to a function you create
dataz = ['potential_temperature', 'salinity', 
         'cdom', 'chlorophyll_a', 'beta_700nm']


# this is a dict to hold our gridded stuff
# until we make a dataset later
d2 = {}

# loop on the variables you want to bin
for varz in dataz:    
    values = ds[varz] # grab this data
    
    #this thing below bins the data
    ret = stats.binned_statistic_2d(ds.index.values, ds.depth, 
                                    values, statistic='mean', bins=[ turns, zgrd ])
    d2[varz] = ret.statistic.T
    
d2.keys()

In [ ]:
# make a quick plot of the results
plt.pcolormesh(d2['potential_temperature'] )
# plt.xlim([0, 40])

## next: bin some '1D' stuff
these are things that dont have a depth dimention:
like, time, lat, lon, u, v, ...

In [ ]:
# things to bin in the x direction
# these also prob need defn in the function call
oneDvars = ['latitude','longitude', 'time', 'u', 'v']

# NB: u, v only have one value per dive sequence, so only half the number profiles!
# actually, its weirder than that... not sure there are more than half...

# dict to hold our 1d bins
d1 = {}

# loop on 1d stuff:
for thing in oneDvars:    
    # time variable needs to be treated a little differently
    if thing == 'time':
        bin_means, bin_edges, binnumber = stats.binned_statistic(ds.index.values,
                    ds[thing].astype(int), statistic = 'mean', bins=turns)
        bin_means = pd.to_datetime(bin_means)
        
    # if we aren't binning time, things are normal
    else:
    
        bin_means, bin_edges, binnumber = stats.binned_statistic(ds.index.values,
                    ds[thing].values, statistic = np.nanmean, bins=turns)
    d1[thing] = bin_means
    
d1.keys()

In [ ]:
# check out work
plt.plot(d1['time'], d1['u'],'.')

plt.figure()
plt.quiver(d1['longitude'], d1['latitude'], d1['u'], d1['v'])

In [ ]:
# how many non-nan u and v's?
# this is confusing
# also, this is not a main issue right now, but needs attention
print( np.count_nonzero( ~np.isnan(d1['v']) ) , ' non-nan velocites')

# this plot shows us that the velocity values are reported near the surface 
plt.plot(interpd.values, ds.u.values, '.')

# put it all together

Now we can add these things together into an xarray dataset that will be easier to work with.

We've created 2D data, and some dims / coords to go with the data

In [ ]:
# need the depth grid centers
zgrd_ctr = zgrd[:-1] + np.diff(zgrd).mean()/2

# create the dataset
ds_gridded = xr.Dataset( coords = {
                           'date': d1['time'].values,
                           'depth': zgrd_ctr ,
                           'lat': ('date', d1['latitude']),
                           'lon': ('date', d1['longitude'])
                          },
               data_vars = {'u': ('date', d1['u']), 
                           'v': ('date', d1['v'])})

# add the other data in a loop
# for varz in dataz:
for varz in d2.keys():
    ds_gridded[varz] = ( ('depth', 'date'),d2[varz] )

    
# thie line below will save the netcdf if you want to work with it in another notebook    
# ds_gridded.to_netcdf('./glider_gridded.nc')

ds_gridded


In [ ]:
# make a simple plot
ds_gridded.potential_temperature.plot( yincrease=False)

# make a section plot
this should be easier now, but this belongs in another notebook

In [ ]:
# can we select a time?
section = ds_gridded.sel( date = 
                         slice('2020-10-18T13:04:29.446530048','2020-10-20T07:42:28.419889920' ) )

plt.figure()
plt.plot(ds_gridded.lon, ds_gridded.lat,'.')
plt.plot(section.lon, section.lat,'.')

plt.figure()
section.potential_temperature.plot(yincrease=False)